In [1]:
 #-----------------------setup database -----------------------------------------
database_username ="root"
database_password ="lalaland"
database_ip ="localhost"
database_name ="pro_database"


In [2]:
import pandas as pd
import numpy as np
import re
import time
import mysql.connector
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib qt
import threading

In [3]:
class YorkSqlTool():
    #===========  SQL TOOLS to make life easier ====================================
    def __init__(self):
        pass
    # Drop database table if exists
    def sqlDropTable(self,cursor, name):
        sql = "DROP TABLE IF EXISTS %s" %name
        cursor.execute(sql)
    
    #Show all the table
    def sqlShowTable(self,cursor):
        k = cursor.execute("SHOW TABLES")
        for x in k:
            print(x)
        
    #Create a table    
    def sqlCreateTable(self,cursor, tableName, dataType):
        cursor.execute("CREATE TABLE %s %s" %(tableName, dataType))
    
    #Drop all data in db
    def sqlDropAllTable(self,cursor):
        k = cursor.execute("SHOW TABLES")
        for x in k:
            self.sqlDropTable(self,cursor,x[0])
    
    #Set foreignkey in table
    def sqlSetForeignKey(self,cursor, Table, key, refTable, refKey):
        sql = "ALTER TABLE %s ADD FOREIGN KEY (%s)\
                REFERENCES %s(%s);"%(Table, key, refTable, refKey)
        cursor.execute(sql)
    
    #drop database if it already that and create new one
    def sqlResetDatabase(self,cursor,dbName):
        cursor.execute("DROP DATABASE IF EXISTS %s"%dbName)
        cursor.execute("CREATE DATABASE %s"%dbName)
        cursor.execute("USE %s"%dbName)
        
    # Line by line insert    
    def sqlInsertData(self,dbEngine, table, keys, primaryKey, dataFrame, update = False):
        val = "%s,"*len(keys) # how many needed to be uploaded
        val = val[0:-1]     # remove the , at the end in list
        
        col = ""
        
        for i in keys:
            col = col +"`%s`, " %i 
        col = col[0:-2] 
        if update:
            pass
        else:
            sql = "INSERT INTO %s (%s) VALUES (%s)\
                ON DUPLICATE KEY UPDATE \
                `%s`= VALUES(`%s`);"%(table,col, val, primaryKey, primaryKey)
        for index,row in dataFrame.iterrows():
            value = row
            dbEngine.execute(sql,value)
    #==================== This is kind of a cheat ===========================
    #pandas to sql is much faster than line by line upload but it does not suport 
    #upsert so this block of code is a work around by creating a temp table that
    #muches the final table and simply merge them and if entries already exit, it will 
    #simply pass, we can create an change the update statment if update is required later
    
    #Mergin temp table to target table
    def sqlInsertTempTable(self,cursor, Table, TempTable, primaryKey):
        
        sql = "INSERT INTO %s SELECT * FROM %s\
             ON DUPLICATE KEY UPDATE\
             `%s`= VALUES(`%s`);"\
             %(Table, TempTable, primaryKey, primaryKey)
        cursor.execute(sql)
    
        
    #upload data to a temp table in the database
    def sqlUploadPandas(self, cursor, df, keys):
        self.sqlDropTable(cursor, "temp")
        tempDataInventroy = df.loc[:,keys] 
        tempDataInventroy =tempDataInventroy.drop_duplicates().astype(str)
        tempDataInventroy.to_sql(name='temp',con=cursor, if_exists="append",\
                          index = False ,chunksize=50)
    #Function to enable and disable foreign key check to speed up upload if needed    
    def sqlDisableFkey(self,dbEngine):
            sql ="SET foreign_key_checks = 0"
            dbEngine.execute(sql) 
    def sqlEnbleFkey(self,dbEngine):
            sql ="SET foreign_key_checks = 1"
            dbEngine.execute(sql)     




In [4]:
class YorkDb(YorkSqlTool):
    dbUsername = None
    dbPassword = None
    dbIp       = None
    dbName     = None
    dbEngine   = None
     
    #============= setup the required items for db ====================================
    def __init__(self,dbUsername, dbPassword, dbIp, dbName):
        self.dbUsername = dbUsername,
        self.dbPassword = dbPassword
        self.dbIp = dbIp
        self.dbName = dbName
        super().__init__()
      
    #============ start connection to db =============================================   
    def connect(self):
        try:
            self.dbEngine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}'.
                    format(database_username, database_password, 
                    database_ip))
            #check if db exist if not make one 
            self.dbEngine.execute("CREATE DATABASE IF NOT EXISTS %s"%database_name)
            self.dbEngine.execute("USE  %s" %database_name)
            #check if tables are correct if not make new ones
            tableCount = self.dbEngine.execute("SELECT COUNT(DISTINCT `table_name`)\
                                            FROM `information_schema`.`columns` \
                                            WHERE `table_schema` = '%s'"%database_name)

            k = tableCount.fetchall()[0][0]
            if k < 2:
                self.sqlResetDatabase(self.dbEngine,database_name)
                self.createCustomerTables()                  

        except:
            print("Connection Failed")
            return False
        


        return True   
    
    #=========== create schema for  this db ==========================================
    """This create all the tables to store the data for sql server
    The function takes one sql engine object and create the reqired table within
    the database"""
   
    def createCustomerTables(self):
        if self.dbEngine ==None:
            self.connect()
        #As we creating a complete new tables, we are going to drop the old DB   
        self.sqlResetDatabase(self.dbEngine,database_name)
        self.sqlShowTable(self.dbEngine)

        #=========== start creating tables ===================
        #************ dataInventory file *********************
        #------------- owner table ===========================
        
        ownerTableType = """
            (
              `OWNER ID`      VARCHAR(16) PRIMARY KEY,\
              `OWNER NAME`    VARCHAR(128),\
              `OWNER ADDRESS` VARCHAR(128),\
              `OWNER CITY`    VARCHAR(128),\
              `OWNER STATE`   VARCHAR(128),\
              `OWNER ZIP`     VARCHAR(16)
            );
          """
        self.sqlCreateTable(self.dbEngine  , "owner", ownerTableType)
    #================ program ===========================    
        programTableType = """
            (`RECORD ID` VARCHAR(16) PRIMARY KEY,
            ` PROGRAM NAME` VARCHAR(128),\
            `PROGRAM ELEMENT (PE)` VARCHAR(16),
            `FACILITY ID` VARCHAR(16)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "program", programTableType)
        
    #================ program element ===========================
        progElemTableType = """
            (`PROGRAM ELEMENT (PE)` VARCHAR(16) PRIMARY KEY,
            `PE DESCRIPTION` VARCHAR(128),\
            `SEAT` VARCHAR(64)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "program_elem", progElemTableType)
    #================= facility ================================
        facilityTableType = """
            (`FACILITY ID`       VARCHAR(16) PRIMARY KEY,
            `FACILITY NAME`      VARCHAR(128),
            `FACILITY ADDRESS`   VARCHAR(128),
            `FACILITY CITY`      VARCHAR(128),
            `FACILITY  STATE`    VARCHAR(128),
            `FACILITY ZIP`       VARCHAR(40),
            `Location`           VARCHAR(64),
            `Census Tracts 2010` VARCHAR(128),
            `2011 Supervisorial District Boundaries (Official)`\
                VARCHAR(128),
            `Board Approved Statistical Areas` VARCHAR(128),
            `OWNER ID` VARCHAR(16)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "facility", facilityTableType)
    
    #================ End of dataInventory file ===============
    
    #************ Inspections file *********************
    #================= inspections ================================
        inspectTableType = """
            (`SERIAL NUMBER`  VARCHAR(16) PRIMARY KEY,
            `ACTIVITY DATE`  DATE,\
            `OWNER ID`       VARCHAR(16),
            `FACILITY ID`    VARCHAR(16),
            `RECORD ID`      VARCHAR(16),
            `PROGRAM STATUS` VARCHAR(16),
            `SERVICE CODE`   VARCHAR(16),
            `SCORE`          FLOAT(8),
            `GRADE`          VARCHAR(8),
            `EMPLOYEE ID`    VARCHAR(16)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "inspections", inspectTableType)
    
    #=============== Service ============================
        serviceTableType = """
            (`SERVICE CODE`  VARCHAR(16) PRIMARY KEY,
            `SERVICE DESCRIPTION`  VARCHAR(128)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "service", serviceTableType)
    
    #================ End of Inspections file ===============
    
    #************ violation file *********************
    #================= violation ================================
        violationTableType = """
            (`SERIAL NUMBER`           VARCHAR(16),
            `VIOLATION  STATUS`        VARCHAR(128),
            `VIOLATION CODE`           VARCHAR(16),
            `VIOLATION DESCRIPTION`    VARCHAR(128),
            `POINTS`                   FLOAT(16)
            )
            """
        self.sqlCreateTable(self.dbEngine  , "violation", violationTableType)
        
    
    #================= add foreign keys =========================
    #================= Programs ================================
        self.sqlSetForeignKey(self.dbEngine  , "facility", "`OWNER ID`",\
                         "owner", "`OWNER ID`")
        self.sqlSetForeignKey(self.dbEngine  , "program", "`PROGRAM ELEMENT (PE)`",\
                         "program_Elem", "`PROGRAM ELEMENT (PE)`")  
        self.sqlSetForeignKey(self.dbEngine  , "program", "`FACILITY ID`",\
                         "facility", "`FACILITY ID`") 
    
    #================= Inspections ==============================
        self.sqlSetForeignKey(self.dbEngine  , "inspections", "`SERVICE CODE`",\
                     "service", "`SERVICE CODE`")    
        self.sqlSetForeignKey(self.dbEngine  , "inspections", "`RECORD ID`",\
                     "program", "`RECORD ID`")
        self.sqlSetForeignKey(self.dbEngine  , "inspections", "`OWNER ID`",\
                     "owner", "`OWNER ID`") 
        
        self.sqlSetForeignKey(self.dbEngine  , "inspections", "`FACILITY ID`",\
                 "facility", "`FACILITY ID`") 
    
    #function return dataFrame with the desired list of data where duplicates are removed
    #================= this end the schema  ===========================================
    
    """
    Because the schema fixes the table structure this part load the three
     different data sets.
    """
    
    #function return dataFrame with the desired list of data where duplicates are removed
    def removeDup(self,dataFrame,args):
        newDataFrame = dataFrame[args]
        return newDataFrame.drop_duplicates().astype(str)
    #==================== Upload Inventroy files ============================
    
    def uploadSqlInventroy(self, dataInventroy):
        start = time.time()
        #connect to dp if there is no connection
        if self.dbEngine ==None:
            self.connect()
        #spliting Inventroy data into their respective dataframe and remove any dupication
        #========================================================================
        keyOwner = ["OWNER ID", "OWNER NAME",\
                    "OWNER ADDRESS", "OWNER CITY",\
                    "OWNER STATE", "OWNER ZIP"]
    
        dataOwner = self.removeDup(dataInventroy, keyOwner)
        
        #load data to database
        
        self.sqlUploadPandas(self.dbEngine, dataInventroy, keyOwner)
        self.sqlInsertTempTable(self.dbEngine,'owner', 'temp', "OWNER ID")
        
        #Program element
        keyProgElem = ["PROGRAM ELEMENT (PE)",\
                       "PE DESCRIPTION",\
                       "SEAT"]
        
        #Program element is an special case where there are only 24 of PE and PE DESCRIPTION and seat
        # are always the same.
        
        #dataProgElem = dataInspections[keyProgElem].drop_duplicates(subset="PROGRAM ELEMENT (PE)")
        self.sqlUploadPandas(self.dbEngine, dataInventroy,  keyProgElem)
        self.sqlInsertTempTable(self.dbEngine,'program_elem', 'temp', "PROGRAM ELEMENT (PE)")
        
        #Facility ----------------------------------------------------------
    
        keyFacility = ["FACILITY ID",'FACILITY NAME',\
                                   'FACILITY ADDRESS','FACILITY CITY',\
                                   'FACILITY  STATE', 'FACILITY ZIP',
                                    'Location', 'Census Tracts 2010',\
                                    '2011 Supervisorial District Boundaries (Official)',
                                    'Board Approved Statistical Areas',\
                                     'OWNER ID']
    
        self.sqlUploadPandas(self.dbEngine, dataInventroy,  keyFacility)
        self.sqlInsertTempTable(self.dbEngine,'facility', 'temp', "FACILITY ID")
        
        #===================  Program ======================================== 
    
        keyProgram = ['RECORD ID', ' PROGRAM NAME',\
                                     'PROGRAM ELEMENT (PE)',"FACILITY ID"]
        self.sqlUploadPandas(self.dbEngine, dataInventroy, keyProgram)
        self.sqlInsertTempTable(self.dbEngine,'program', 'temp', "RECORD ID")
    
        print ("Inventroy loaded in %.2f" %(time.time()-start))
        
        
#==================== End of inventroy ============================================================

    def uploadSqlInspections(self,dataInspections):
        start = time.time()
        #connect to dp if there is no connection
        if self.dbEngine ==None:
            self.connect()
        
        #==========================================================================
        #spliting Inspections data into their respective dataframe
        #dropping all data that are dupication bettween inspections.csv and program.csv
        #========================================================================
        
        keyProgramService = ['SERVICE CODE', 'SERVICE DESCRIPTION']
    
        self.sqlUploadPandas(self.dbEngine, dataInspections, keyProgramService)
        self.sqlInsertTempTable(self.dbEngine,'service', 'temp', "SERVICE CODE")
         
        #=== There are more data in inspection than inventory so the owner table need to be updated
        #=== with the latest data
    
    
        keyInspectOwner = ['OWNER ID', 'OWNER NAME']
        inspectOwner = self.removeDup(dataInspections, keyInspectOwner)
        
        keyOwner = ["OWNER ID", "OWNER NAME",\
                    "OWNER ADDRESS", "OWNER CITY",\
                    "OWNER STATE", "OWNER ZIP"]
    
        inspectOwner = inspectOwner.reindex(columns = keyOwner )
        
        #=== fill missing data with None, as the schema reqired
        inspectOwner = inspectOwner.where(pd.notnull(inspectOwner), None)
        self.sqlUploadPandas(self.dbEngine, inspectOwner, keyOwner)
        self.sqlInsertTempTable(self.dbEngine,'owner', 'temp', "OWNER ID")
        
        #============ have to do the same for facility but faci
    
        keyFacility = ["FACILITY ID",'FACILITY NAME',\
                                   'FACILITY ADDRESS','FACILITY CITY',\
                                   'FACILITY  STATE', 'FACILITY ZIP',
                                    'Location', 'Census Tracts 2010',\
                                    '2011 Supervisorial District Boundaries (Official)',
                                    'Board Approved Statistical Areas',\
                                     'OWNER ID']
        keyInspectFacility = ["FACILITY ID",'FACILITY NAME',\
                                   'FACILITY ADDRESS','FACILITY CITY',\
                                    'FACILITY ZIP',
                                    'Location', 'Census Tracts 2010',\
                                    '2011 Supervisorial District Boundaries (Official)',
                                    'Board Approved Statistical Areas',\
                                     'OWNER ID']
    
        inspectFacility = self.removeDup(dataInspections, keyInspectFacility)
    
        inspectFacility = inspectFacility.reindex(columns = keyFacility )
    
        #=== fill missing data with None, as the schema reqired
        inspectFacility = inspectFacility.where(pd.notnull(inspectFacility), None)
    
        
        self.sqlUploadPandas(self.dbEngine, inspectFacility, keyFacility)
        self.sqlInsertTempTable(self.dbEngine,'facility', 'temp', "FACILITY ID")
        
        #==================== program element =========================
        keyProgElem = ["PROGRAM ELEMENT (PE)",\
                   "PE DESCRIPTION",\
                   "SEAT"]
        
        #Program element is an special case where there are only 18 of PE and PE DESCRIPTION and seat
        # are always the same.
            
        #dataProgElem = dataInspections[keyProgElem].drop_duplicates(subset="PROGRAM ELEMENT (PE)")
        dataProgElem = dataInspections.dropna()
        
        
        self.sqlUploadPandas(self.dbEngine,  dataProgElem ,  keyProgElem )
        self.sqlInsertTempTable(self.dbEngine,'program_elem', 'temp', "PROGRAM ELEMENT (PE)")
       
         #===================  Program ======================================== 
    
        keyProgram = ['RECORD ID', ' PROGRAM NAME',\
                                     'PROGRAM ELEMENT (PE)',"FACILITY ID"]
        keyInspectProgram  = ['RECORD ID',\
                                     'PROGRAM ELEMENT (PE)',"FACILITY ID"]
    
        inspectProgram = self.removeDup(dataInspections,keyInspectProgram )
        
        #=== fill missing data with None, as schema reqired
        inspectProgram = inspectProgram.reindex(columns = keyProgram )
        
        inspectProgram  = inspectProgram.where(pd.notnull(inspectProgram), None)
    
        self.sqlUploadPandas(self.dbEngine, inspectProgram,  keyProgram )
        self.sqlInsertTempTable(self.dbEngine,'program', 'temp', "RECORD ID")
        
         #==============    finally inspection ================================
    
        keyInspection = ['SERIAL NUMBER','ACTIVITY DATE', 'OWNER ID',\
                                       'FACILITY ID', 'RECORD ID','PROGRAM STATUS',\
                                       'SERVICE CODE', 'SCORE', 'GRADE',\
                                       'EMPLOYEE ID']
    
        dataInspect = self.removeDup(dataInspections, keyInspection)
        
        #convert the date to sql date formate
        dataInspect['ACTIVITY DATE'] = pd.to_datetime(dataInspections['ACTIVITY DATE'],\
                                                      format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
    
        #cast the column to number and set null to None
        dataInspect.loc[:,'SCORE'] = pd.to_numeric(dataInspect['SCORE'],\
                                                   errors='coerce')
        
        #change all nan value to something sql can understand
        dataInspect = dataInspect.where(dataInspect.notnull(), None)
        dataInspect =dataInspect.dropna()
        
        self.sqlUploadPandas(self.dbEngine, dataInspect,  keyInspection )
        self.sqlInsertTempTable(self.dbEngine,"inspections", 'temp', "RECORD ID")
        
    
        print ("Inspections loaded in %.2f" %(time.time()-start))
           
#============= End of Inspections =============================================================

    #============ Start of violations ============================================
    def uploadSqlViolations(self,dataViolations):
        start = time.time()
        
        #connect to dp if there is no connection
        if self.dbEngine ==None:
            self.connect()
        #==========================================================================
        #violations.csv does not require spliting. But because it does not 
        #have unique id data base need to be plage of dupicate after update

        dataViolations.drop_duplicates(inplace = True)


        keyviolation = ["SERIAL NUMBER", "VIOLATION  STATUS",\
                         "VIOLATION CODE","VIOLATION DESCRIPTION",
                         "POINTS"]
        #it can be directly upload to db
        dataViolations.to_sql(name='violation',con=self.dbEngine, if_exists='append',\
                          index = False ,chunksize=50)
        #clean up dupicated
        mysql = """CREATE TABLE copy_violation SELECT DISTINCT `SERIAL NUMBER`,\
                    `VIOLATION  STATUS`,`VIOLATION CODE`,
                    `VIOLATION DESCRIPTION`, `POINTS`
                    FROM violation"""
        self.dbEngine.execute(mysql)

        mysql = """DROP TABLE violation;"""
        self.dbEngine.execute(mysql)
        mysql = "ALTER TABLE copy_violation RENAME TO violation;"
        self.dbEngine.execute(mysql)

        print ("Violations loaded in %.2f" %(time.time()-start))

    

In [5]:
#processing part of the program!!!!!!
class YorkProcessing():
    dB = None
    def __init__(self, dbUsername, dbPassword, dbIp, dbName):
        self.dB = YorkDb(dbUsername, dbPassword, dbIp, dbName)
        
        
    #============= this do the reading of the files and clean them as they pass=============
    #It will return data in pandas dataframe
    def readCsvData(self,path, idSwidch = None, encode ="utf-8-sig" ):
        #load and remove Byte Order Mark
        with open(path, encoding=encode) as file:
            rawData = file.readlines()
        data = [] #list to store the data

        #get header
        title = (rawData[0].rstrip("\n").split(","))

        #change thet id if requested in preperation to Mongodb 
        if idSwidch != None:
            for i, k in enumerate (title):
                if idSwidch == k:
                    title[i] = "_id"

        # get the rest of the data line by line
        for g, line in enumerate (rawData[1:-1]):
            line = (line.rstrip("\n"))   # strip off the endline
            #Pattern match to ignore , within doubele quote otherwise split comma 
            tempsplit=re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", line) 
            #Match the column to the data and put it into list of dictionary
            templine = {i:tempsplit[k] for k,i in enumerate (title)}
            data.append(templine)
        #this part extract seat from "PE DESCRIPTION" and put it into an extra dictionary pair
        for k,i in enumerate (data):
            try:
                #make sure PE exist
                if "PE DESCRIPTION" in i:
                    if "(" in i["PE DESCRIPTION"] != None:
                        #find the index between the ()
                        braketPos = list(re.finditer(r'\(.*\)',i["PE DESCRIPTION"], flags = re.IGNORECASE))[0].span()
                        #this make new dictionary Seat
                        #Removing the comma
                        tempSeat = i["PE DESCRIPTION"][braketPos[0]+1:braketPos[1]-1].replace(",","")
                        #correct seats
                        
                        #=============== correct the seats as we read data in ===========================================
                        if "sf" not in tempSeat.lower():
                            if float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])<31 :
                                tempSeat = "0-30"
                            elif float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])<61:
                                tempSeat = "31-60"
                            elif float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])<151:
                                tempSeat = "61-150"
                            elif float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])<1999:
                                tempSeat = "150 +"
                            elif float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])>1998:
                                tempSeat = "1-1,999 SF"
                            else:
                                tempSeat = "2,000+ SF"
                        else:
                            if float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])<2000:
                                tempSeat = "1-1,999 SF"
                            if float(re.findall(r"(\d+)(?!.*\d)", tempSeat ) [0])>1999:
                                tempSeat = "2,000+ SF"
                        
                        #write seat to dictionary ======================================================================    
                        data[k].update({"SEAT":tempSeat})

                        #Join the rest backup
                        newPE = (i["PE DESCRIPTION"][:braketPos[0]]+i["PE DESCRIPTION"][braketPos[1]:]).replace("SEATS","")
                        data[k].update({"PE DESCRIPTION":newPE})
                    else:
                        #If no () set seat to nan
                        data[k].update({"SEAT":np.nan})
            except Exception as e:
                print(e)
        df = pd.DataFrame(data)
        return df
    #==========Read the first column and decide which data file it is
    
    def dataToDb(self,dataFrame):
        try:
            #Inventroy
            if dataFrame.columns[0] == 'FACILITY ID':
            
                self.dB.uploadSqlInventroy(dataFrame)
                print("inventroy")
            #Violation
            elif dataFrame.columns[0] == 'SERIAL NUMBER':
                self.dB.uploadSqlViolations(dataFrame)
                print("violation")
            #Inspections
            elif dataFrame.columns[0] == 'ACTIVITY DATE':
                self.dB.uploadSqlInspections(dataFrame)
                print("inspections")
            else:
                print("Wrong file")
                return False
        except():
            return False
        return True
    
    #============== just getting a connection to our SQL server =================================  
    def connectToDb(self):
        return self.dB.connect()
    def checkConnection(self):
        if self.dB.dbEngine == None:
            self.connectToDb()
            
    #ploting part of the processing
    def statPlot(self, option):
        #ploting option so far "ZIP Code", "Seat Type", violation
        if option == "Seat Type":
            try:
                self.getSeatPlot()
            except:
                print ("Seat plot issuse")
                return False
        elif option =="Zip Code":
            try:
                self.getZipPlot()
            except:
                print ("Zip plot issuse")
                return False
        elif option =="Violation Code":
            try:
                self.vioPlot()
            except:
                print ("vio plot issuse")
                return False
        elif option =="Zip Correlation":
            try:
                self.corrPlot()
            except:
                print ("corr plot issuse")
                return False
            
            
        return True
            
        
    
    #============plot the seat data==================================================
    def getSeatPlot(self):
        #ask the master mind for data aka our all controling sql server
        sql ="""SELECT inspections.`SCORE`, program_elem.`SEAT`
        ,Year(inspections.`ACTIVITY DATE`)
        FROM inspections\
        LEFT JOIN program\
        ON inspections.`RECORD ID` = program.`RECORD ID`\
        LEFT JOIN program_elem\
        ON program.`PROGRAM ELEMENT (PE)` = program_elem.`PROGRAM ELEMENT (PE)`
        WHERE inspections.`PROGRAM STATUS` = "ACTIVE";"""
        try:
            self.checkConnection()
            k = self.dB.dbEngine.execute(sql)
        except:
            return False
        myresult = k.fetchall()
        
        #put data into datframe
        data = pd.DataFrame(myresult,columns= ["SCORE","SEAT", "Date"])
        
        #goup up data and seat and get the stat
        finalData = data.groupby([data.Date, 'SEAT'])['SCORE'].agg(['mean', lambda x:x.mode(), 'median'])
        finalData = finalData.reset_index()
        
        #going to plot this in 4 graphs so spliting the year here
        
        numYear = finalData["Date"].unique()
        #setting up the graph
        fig, ax = plt.subplots(len(numYear),1,figsize=(10,10))
        for i,y in enumerate (numYear):
            #get the data for each year
            year= finalData[finalData["Date"] == y]
            
            #set the x y label
            labels = finalData["SEAT"]
            #generate the x steps
            x = np.arange(len(year["SEAT"]))
            
            # the width of the bars
            width = 0.25 
            
            #plot the data
            ax[i].barh(x - width, year["mean"], width, label='Mean')
            ax[i].barh(x , year["<lambda_0>"], width, label='Mode')
            ax[i].barh(x + width, year["median"], width, label='Median')
            
            #set the x range
            mmin = np.min(year.iloc[:,[2,3,4]].min())
            mmax = np.max(year.iloc[:,[2,3,4]].max())
            ax[i].set_xlim([mmin*0.98, mmax*1.02])
            
            # Add some text for labels, title and custom y-axis
            ax[i].set_ylabel('SEAT')
            ax[i].set_title('%s Score'%y)
            ax[i].set_yticks(x)
            ax[i].set_yticklabels(labels)
            ax[i].legend()
            fig.subplots_adjust(hspace=0.4)
        plt.show(block = False)
        return True
    #============Plot zip data ==========================================================
    def getZipPlot(self):
        
        #ask the goddness of sql to get use zip code score and date
        sql ="""SELECT DATE_FORMAT(`ACTIVITY DATE`,'%Y') as "Year", `SCORE`,
                LEFT(facility.`FACILITY ZIP`,5)
                FROM inspections\
                LEFT JOIN facility\
                ON inspections.`FACILITY ID` = facility.`FACILITY ID`
                WHERE inspections.`PROGRAM STATUS` = "ACTIVE";"""
        try:
            self.checkConnection()
            k = self.dB.dbEngine.execute(sql)
        except:
            return False
        #getting the data into dataframe and label the columns
        myresult = k.fetchall()        
        data = pd.DataFrame(myresult,columns= ["Date","SCORE","FACILITY ZIP"])
        # do the magic here grouping data and zip and calculate the stat 
        finalData= data.groupby([data.Date, "FACILITY ZIP"])['SCORE'].agg(['mean', lambda x:x.mode()[0], 'median'])
        #put things back in columns 
        finalData = finalData.reset_index()
        
        #going to plot this in 4 graphs so spliting the year here
        numYear = finalData["Date"].unique()
        
        #setting up the graph
        fig, ax = plt.subplots(len(numYear),1,figsize=(10,10))
        fig.subplots_adjust(hspace=0.4)

        for i,y in enumerate (numYear):

            year = finalData[finalData["Date"] == y]
            year = year.astype(float)
            #Rebinning the data
            bins = list(range(int(np.min(year["FACILITY ZIP"])),\
                      int(np.max(year["FACILITY ZIP"])),50))
            groups = year.groupby(pd.cut(year["FACILITY ZIP"], bins))

            # Get the mean of b, binned by the values in a
            finalMean = groups.agg([np.mean, np.std]) 

            #==============doing the ploting =============================================
            ax[i].scatter(finalMean["FACILITY ZIP"]["mean"],finalMean["mean"]["mean"],\
                          color="b", marker = "x", label='Mean')
            ax[i].errorbar(finalMean["FACILITY ZIP"]["mean"],finalMean["mean"]["mean"],\
                          xerr=finalMean["FACILITY ZIP"]["std"], yerr = finalMean["mean"]["std"],\
                           fmt='',ls='none', color="b")
            ax[i].scatter(finalMean["FACILITY ZIP"]["mean"],finalMean["<lambda_0>"]["mean"],\
                          color="r", marker = "4", label='Mode')
            ax[i].errorbar(finalMean["FACILITY ZIP"]["mean"],finalMean["<lambda_0>"]["mean"],\
                          xerr=finalMean["FACILITY ZIP"]["std"], yerr = finalMean["<lambda_0>"]["std"],\
                           fmt='',ls='none',color="r")
            ax[i].scatter(finalMean["FACILITY ZIP"]["mean"],finalMean["median"]["mean"],\
                          color="g", marker = "*", label='Median')
            ax[i].errorbar(finalMean["FACILITY ZIP"]["mean"],finalMean["median"]["mean"],\
                          xerr=finalMean["FACILITY ZIP"]["std"], yerr = finalMean["median"]["std"],\
                           fmt='',ls='none',color="g")

            # Add some text for labels, title and custom y-axis
            ax[i].set_ylabel('SCORE')
            ax[i].set_title('%s Zip Code'%y)
            ax[i].legend(loc='lower left')
            mmin = np.min(year.iloc[:,[2,3,4]].min())
            mmax = np.max(year.iloc[:,[2,3,4]].max())
            ax[i].set_ylim([mmin*0.98, mmax*1.02])


        ax[len(numYear)-1].set_xlabel("Zip Code (Bin size 50)")
        plt.show(block = False)
        return True
    #====================Plot violation ========================================
    #again ask for data
    def vioPlot(self):
        sql = """SELECT violation.`VIOLATION CODE` AS code,count(`VIOLATION CODE`) \
              FROM violation\
              LEFT JOIN inspections\
              ON inspections.`SERIAL NUMBER` = violation.`SERIAL NUMBER`\
              WHERE inspections.`PROGRAM STATUS` = "ACTIVE"\
              GROUP BY `VIOLATION CODE`"""
        try:
            self.checkConnection()
            k = self.dB.dbEngine.execute(sql)
        except:
            return False
        myresult = k.fetchall()
        #data into dataframe for processing
        data = pd.DataFrame(myresult,columns= ["V code","Count"])
        #sort the data so it can be plot in in order
        data.sort_values("Count", inplace = True, ascending = False)
        
        #===== work out the step size for 6 graph 
        end = int(len(data["V code"]))
        ss = end/6
        ss = int(ss)
        print (ss)
        start = 0
        #===get ready to plot the data
        fig = plt.figure(figsize=(10,10))
        fig.tight_layout(pad=1)
        #hex coloring =======================================
        colour = ["6930c3","5e60ce","5390d9","48bfe3","64dfdf","72efdd","80ffdb"]
        colour = ["#" + c for c in colour]
        #=========start the ploting
        for j,i in enumerate (range(ss,end+1,ss)):
        
            plt.subplot(3,2,j+1)
            plt.suptitle("Establishment Violations")
            plt.barh(data["V code"][start:i], data["Count"][start:i],color =colour[j])
            #label y 
            start = start + ss
            if j in [0,2,4]:
                plt.ylabel("Violation Code")
            #only put xlabel at the bottom two plots
            if j >3:
                plt.xlabel("Number of active Establishment (Counts)")
        plt.show(block = False)
        return True
    #==================Plot correlation =============================
    def corrPlot(self):
        sql = """SELECT LEFT(facility.`FACILITY ZIP`,5) AS code,\
              count(DISTINCT facility.`FACILITY ID`),\
              count(`VIOLATION CODE`),\
              count(`VIOLATION CODE`)/count(DISTINCT facility.`FACILITY ID`)      
              FROM facility\
              RIGHT JOIN inspections\
                ON inspections.`FACILITY ID` = facility.`FACILITY ID`
              RIGHT JOIN violation\
              ON inspections.`SERIAL NUMBER` = violation.`SERIAL NUMBER`\
              WHERE inspections.`PROGRAM STATUS` = "ACTIVE"\
              GROUP BY LEFT(facility.`FACILITY ZIP`,5)"""
        try:
            self.checkConnection()
            k = self.dB.dbEngine.execute(sql)
        except:
            return False
        myresult = k.fetchall()
        #data into dataframe for processing
        data =pd.DataFrame(myresult, columns= ["ZIP","No Fac","No Vio","v/F"]).astype(float)
        stippedData = data[data["No Fac"]>7]
        removedHigh = stippedData[stippedData["ZIP"]<92000]
        std = removedHigh["v/F"].std()
        mean = removedHigh["v/F"].mean()
        plt.figure()
        plt.scatter(removedHigh["ZIP"],removedHigh["v/F"])
        plt.errorbar(removedHigh["ZIP"],removedHigh["v/F"],yerr = std,ls = "none", fmt="")
        plt.fill_between(removedHigh["ZIP"], (mean-std), (mean+std),alpha = 0.2, label = r'$\sigma$')
        plt.hlines(mean, 89900, 91900, color ="r", label = "Mean") 
        plt.ylabel("Violations per facility")
        plt.xlabel("Zip Code") 
        plt.legend()
        plt.show(block= False)
        return True


In [ ]:
#get processing
y = YorkProcessing(database_username, database_password, database_ip, database_name)
#load data into memory and some small clean up              
pathIns= "DataSet1\\Inspections.csv"
dataInspections = y.readCsvData(pathIns)
pathInv= "DataSet1\\Inventroy.csv"
dataInventroy   = y.readCsvData(pathInv)
pathVio = "DataSet1\\violations.csv"
dataViolations  = y.readCsvData(pathVio)   
y.dataToDb(dataInventroy)
y.dataToDb(dataInspections)
y.dataToDb(dataViolations)

Inventroy loaded in 8.28
inventroy
Inspections loaded in 47.81
inspections
